### Reproduce the Capitanio 2013 analytical model

In this notebook, I reproduced the model from the Capitanio 2013 paper.

The missing data in the paper, W - slab width, l - plate length, and $\Delta \rho$ are figured out from a MCMC algorithm and a fit to their figure 4.

$l = 2000 km$

$W = 180 km$

$\Delta \rho = 70 km/m^3$

In the notebook, I replot their figure 4 with nearly identical results

Following these steps, I wrap their model into a single class which could be imported to other scripts

In [ ]:
import os, sys
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import gridspec, cm

year = 365.0 * 24 * 3600.0
ma = 1e6 * year # convert ma to s
cm = 1e-2 # cm to m
cm_per_yr = cm / year # cm/yr to m/s


In [ ]:
class CAP13():
    '''
    class for the Capitanio 13 paper
    '''
    def __init__(self, **kwargs):
        # default values
        self.rho_0 = kwargs.get('rho_0', 3000.0) # Reference density
        self.rho_w = kwargs.get('rho_w', 1000.0) # Water density
        self.alpha = kwargs.get('alpha', 1e-5) # Thermal expansivity
        self.kappa = kwargs.get('kappa', 1e-6) # Thermal diffusivity
        self.delta_T = kwargs.get('delta_T', 1300.0) # Temperature difference
        self.g = kwargs.get('g', 9.81) # Gravity Acceleration
        self.eta_M = kwargs.get('eta_M', 10**21.0) # Upper mantle viscosity
        self.eta_L = kwargs.get('eta_L', 1e4*self.eta_M) # Lithospheric viscosity, by ratio
        self.eta_A = kwargs.get('eta_A', 3e-2*self.eta_M) # Asthenosphere viscosity
        self.delta_rho = kwargs.get('delta_rho', 80.0) # density constract between slab and mantle, not given in the paper
        self.h_A = 500e3 # Asthenosphere thickness

    def CalculateCoefficents(self):
        '''
        compute the coefficents for forces used in this solution
        '''
        # buoyancy force
        # plate thickness
        # h = 2.32 * (kappa * t)**0.5
        # length of the slab to slab dept
        # L = h_D * np.arcsin(phi)
        # buoyancy force to slab dimensions
        # F_B = delta_rho * self.g * L * h
        # F_B = c1 * h_D * np.arcsin(phi) * t**0.5
        c1 = 2.32 * self.delta_rho * self.g * self.kappa**0.5
        
        # ridge push force, from analytical solution
        # F_RP = c2 * t
        c2 = self.g * self.delta_rho * self.kappa * (1.0 + 2.0 * self.delta_rho / np.pi / (self.rho_0 - self.rho_w))
        
        # resisting force
        # the drap coefficient, depend on the two lengths of a ellipsoid of W and L
        # K_M = 24 * 2**0.5 * (1 + np.log(W/L))
        # the resisting force, this includes the shear drag and the anchoring force as the two components
        # F_MD = -c3 * (1 + np.log(W/L)) * U
        c3 = 24 * 2**0.5 * self.eta_M
        
        # the athenospher contour flow resistance
        # tau_A = -2 * eta_A * u / h_A * (2.0 + 3.0 * h / h_A)
        # F_AD = tau_A * l
        # F_AD = -c4 * u * l
        # u: velocity of the plate; l: length of the plate
        # c4 = 2 * eta_A / h_A * (2.0 + 3.0 * h / h_A)
        # assume average values for h / h_A = 0.2
        c4 = 2 * self.eta_A / self.h_A * (2.0 + 3.0 / 5.0)
        
        # resistance to bending
        # minimum bending radius is
        # R = 0.5 * h**3.0 (eta_L / eta_M)**0.5
        # the bending force to radius is
        # F_BR = - 2.0 / 3.0 * U * eta_L * (h / R) **3.0
        # the beding force to velocity is
        # F_BR = - 16.0 / 3.0 * (eta_M / eta_L)**0.5 * eta_M * U
        # assume constant ratio between lithosphere and mantle
        # note here the assumption is eta_M / eta_L = 1.0; Question: is this right?
        # F_BR = -c5 * U
        c5 = 16.0 * self.eta_M / 3.0

        # assign this to a dict
        coeffs = {'c1': c1, 'c2': c2, 'c3': c3, 'c4': c4, 'c5': c5}

        return coeffs
    
    def CalculateVelocities(self, l, t, phi, W, h_D):
        '''
        Calculate the sinking rate, plate sliding, trench retreat velocity and the critical age
        Inputs:
            (all in U/I)
            phi: dipping angle
            l: plate length
            t: plate age
            W: slab width
            L: slab length
            h_D: slab depth
        '''
        # first get the model coefficents
        coeffs = self.CalculateCoefficents()
        c1 = coeffs['c1']
        c2 = coeffs['c2']
        c3 = coeffs['c3']
        c4 = coeffs['c4']
        c5 = coeffs['c5']

        # slab buoyancy is balanced by viscous resistance and bending resistance
        # F_B + F_MD + F_BR = 0.0 -> sinking rate
        L = h_D / np.sin(phi) # length of the slab
        v = c1 * h_D * np.arcsin(phi) * t**0.5 / (c5 - c3 * (1 + np.log(W/L)))

        # ridge push is balanced by the athenospheric drag -> sliding velocity
        # l: length of the plate
        u = c2 * t / c4 / l

        # from the geometric relation (in their eq 2 and 5)
        u_T_A = c1 * h_D * np.cos(phi) * np.sin(phi)**(-2.0) * t**0.5
        u_T_B = c5 - c3 * (1 + np.log(W/L))
        u_T_C = c2 * t / c4 / l
        u_T =  u_T_A / u_T_B - u_T_C

        # a critical age, between trench advance and retreat (eq 6)
        t_c_A = c4 * l / c2
        t_c_B = (c1 * h_D * np.cos(phi) * np.sin(phi)**(-2.0))
        t_c_C = (c5 - c3 * (1.0 + np.log(W/L)))
        t_c = (t_c_A * t_c_B / t_c_C)**2.0

        results = {'v': v, 'u': u, 'u_T': u_T, 't_c': t_c}

        return results, coeffs

### reproduce their figure 3

The values for $\Delta \rho$, l and W are from the MCMC algorithm down below
With that, I generated very good fit to their fig3

In [ ]:
# reproduce their figure 3
Cap13 = CAP13(eta_M=1e21, delta_rho=70.0)
Cap13_1 = CAP13(eta_M=5e20, delta_rho=70.0)
Cap13_2 = CAP13(eta_M=5e21, delta_rho=70.0)
l = 2000e3
W = 180e3
ts = np.linspace(0.0, 160.0, 100) * ma
# 1e21 mantle viscosity
results_57_660, coeffs_57_660 = Cap13.CalculateVelocities(l, ts, 57.0 * np.pi / 180.0, W, 660e3)
results_85_110, coeffs_85_110 = Cap13.CalculateVelocities(l, ts, 85.0 * np.pi / 180.0, W, 110e3)
results_43_1386, coeffs_43_1386 = Cap13.CalculateVelocities(l, ts, 43.0 * np.pi / 180.0, W, 1386e3)
results_33_2310, coeffs_33_2310 = Cap13.CalculateVelocities(l, ts, 33.0 * np.pi / 180.0, W, 2310e3)
# 5e20 mantle viscosity
results_57_660_1, coeffs_57_660_1 = Cap13_1.CalculateVelocities(l, ts, 57.0 * np.pi / 180.0, W, 660e3)
# 5e20 mantle viscosity
results_57_660_2, coeffs_57_660_2 = Cap13_2.CalculateVelocities(l, ts, 57.0 * np.pi / 180.0, W, 660e3)

fig = plt.figure(tight_layout=True, figsize=(15, 6))
gs = gridspec.GridSpec(1, 2)

ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[0, 1])
ax1.plot(ts/ma, (results_57_660['u_T'] + results_57_660['u'])/cm_per_yr, 'b', label="hinge retreat")
ax1.plot(ts/ma, results_57_660['u']/cm_per_yr, 'r', label="sliding velocity")
ax1.plot(ts/ma, results_57_660['u_T']/cm_per_yr, 'k', label="net trench velocity")
# ax1.plot(ts/ma, results_85_110['u']/cm_per_yr, 'r--', label="sliding velocity")
ax1.plot(ts/ma, (results_85_110['u_T'] + results_85_110['u'])/cm_per_yr, 'b--', label="hinge retreat")
ax1.plot(ts/ma, results_85_110['u_T']/cm_per_yr, 'k--', label="net trench velocity")
ax1.plot(ts/ma, results_43_1386['u_T']/cm_per_yr, 'k', linestyle="dotted", label="net trench velocity")
ax1.plot(ts/ma, results_33_2310['u_T']/cm_per_yr, 'k', linestyle="dashdot", label="net trench velocity")
ax1.set_ylim((-20.0, 20.0))
ax1.grid()

ax2.plot(ts/ma, results_57_660['u_T']/cm_per_yr, 'k', label="net trench velocity")
ax2.plot(ts/ma, results_57_660['u']/cm_per_yr, 'r', label="sliding velocity")
ax2.plot(ts/ma, results_57_660_1['u_T']/cm_per_yr, 'k--', label="net trench velocity")
ax2.plot(ts/ma, results_57_660_1['u']/cm_per_yr, 'r--', label="sliding velocity")
ax2.plot(ts/ma, results_57_660_2['u_T']/cm_per_yr, 'k', linestyle="dotted", label="net trench velocity")
ax2.plot(ts/ma, results_57_660_2['u']/cm_per_yr, 'r', linestyle="dotted", label="sliding velocity")
ax2.set_ylim((-20.0, 20.0))
ax2.grid()

# print(results)

In [ ]:
print(results_57_660['t_c'])

### Below I use a MCMC algorithm to fit the presented results in the paper

I took the data from their figure 3 a, the solid case ($\phi=57^o$, $h_D=660 km$)

In [ ]:
import numpy as np
import pymc as pm
import arviz as az
from matplotlib import pyplot as plt
from matplotlib import gridspec

# define a function for using delta_density and l as free variables
def Foo0(ts, delta_rho, l, W):

    year = 365.0 * 24 * 3600.0
    ma = 1e6 * year # convert ma to s
    cm = 1e-2 # cm to m
    cm_per_yr = cm / year # cm/yr to m/s
    
    # reproduce their figure 3
    Cap13 = CAP13(eta_M=1e21, delta_rho=delta_rho)
    phi = 57.0 * np.pi / 180.0
    h_D = 660e3
    # l = 2500e3
    # W = 80e3
    # ts = np.linspace(0.0, 160.0, 100) * ma
    results, coeffs = Cap13.CalculateVelocities(l, ts, phi, W, h_D)
    return results['u_T'], results['u']

def Foo1(ts, delta_rho, l, W):

    year = 365.0 * 24 * 3600.0
    ma = 1e6 * year # convert ma to s
    cm = 1e-2 # cm to m
    cm_per_yr = cm / year # cm/yr to m/s
    
    # reproduce their figure 3
    Cap13 = CAP13(eta_M=1e21, delta_rho=delta_rho)
    phi = 33.0 * np.pi / 180.0
    h_D = 2310e3
    # l = 2500e3
    # W = 80e3
    # ts = np.linspace(0.0, 160.0, 100) * ma
    results, coeffs = Cap13.CalculateVelocities(l, ts, phi, W, h_D)
    return results['u_T'], results['u']

ts_ma_0a = [0.20085777603906552, 3.0129126139839237, 7.230987208668217, 12.05165045593571, 16.671440602698105, 26.714482649311194, 33.543754305901295, 48.206601823742794, 65.07891552694596, 84.16069541551086, 102.2381710994144, 126.74320288782991, 126.74320288782991, 140.60258865258314, 157.2740445797473]
u_T_cm_yr_0a = [0.0638288132687812, 0.8297867470599201, 1.0212756217794166, 0.9574468085106336, 0.8936179952418488, 0.5744690590716388, 0.12765519162441308, -0.6382978723404289, -1.6595734941198543, -2.936169238800705, -4.276596231663486, -6.127658600502832, -6.127658600502832, -7.2127703402904775, -8.48936608497133]

ts_ma_0b = [0.8034464286222519, 10.444757598691206, 52.02296086634899, 159.68436854114802]
u_cm_yr_0b = [0.0638288132687812, 1.085102000135052, 5.36170310162483, 16.531915867582278]

ts_ma_1a = [0.6025733281171822, 3.0129126139839237, 9.842184270574023, 21.090388297887483, 33.94548518244543, 51.01865666168766, 62.86943401711828, 75.32281534959813, 95.20804166678522, 107.25969212272095, 124.93543693008034, 137.99139159067738, 149.84219959503997, 158.88093743699181]
u_T_cm_yr_1a = [1.2765957446808436, 3.638293489496757, 5.936169725783326, 8.04255465243725, 9.510639271837594, 10.595744924342373, 11.170213983414019, 11.553191732853016, 11.936170699748587, 11.999998295560793, 11.999998295560793, 11.872340669023227, 11.744681825029087, 11.680849359390578]


N_obs = 30
ts_ma = np.linspace(1.0, 155.0, N_obs)
ts = ts_ma * ma
u_T_cm_yr_0 = np.interp(ts_ma, ts_ma_0a, u_T_cm_yr_0a)
u_cm_yr_0 = np.interp(ts_ma, ts_ma_0b, u_cm_yr_0b)
u_T_cm_yr_1 = np.interp(ts_ma, ts_ma_1a, u_T_cm_yr_1a)

with pm.Model() as model:
    # define prior 
    delta_rho = pm.Normal("delta_rho", mu=80.0, sigma=40.0)
    l = pm.Normal("l", mu=3000e3, sigma=1000e3)
    W = pm.Normal("W", mu=80e3, sigma=40e3)
    sigma = pm.HalfNormal("sigma", sigma=2.0)

    simulated_u_T_0, simulated_u_0 = Foo0(ts, delta_rho, l, W)
    simulated_u_T_1, simulated_u_1 = Foo1(ts, delta_rho, l, W)
    diff_u_T_0 = u_T_cm_yr_0 - simulated_u_T_0 / cm_per_yr
    diff_u_0 = u_cm_yr_0 - simulated_u_0 / cm_per_yr
    diff_u_T_1 = u_T_cm_yr_1 - simulated_u_T_1 / cm_per_yr
    
    # diff_combined = pm.math.maximum(pm.math.abs(diff_u_T_0), pm.math.abs(diff_u_0))
    # diff_combined = pm.math.maximum(diff_combined, pm.math.abs(diff_u_T_1))
    # diff_combined = pm.math.maximum(pm.math.abs(diff_u_T_0/u_T_cm_yr_0), pm.math.abs(diff_u_0/u_cm_yr_0)) # use the minimum of all ratios
    diff_combined = pm.math.maximum(pm.math.abs(diff_u_T_0/u_T_cm_yr_0), pm.math.abs(diff_u_0/u_cm_yr_0*3.0)) # add weight to the sliding velocity
    diff_combined = pm.math.maximum(diff_combined, pm.math.abs(diff_u_T_1/u_T_cm_yr_1))

    # likelihood = pm.Normal('likelihood', mu=simulated_u_T/cm_per_yr, sigma=sigma, observed=u_T_cm_yr_0)
    # likelihood = pm.Normal('likelihood', mu=diff_u_T_0, sigma=sigma, observed=np.zeros(N_obs))
    likelihood = pm.Normal('likelihood', mu=diff_combined, sigma=sigma, observed=np.zeros(N_obs))

    # Use the Metropolis-Hastings algorithm
    step = pm.Metropolis()

with model:
    # Sample from the posterior
    trace_for_summary = pm.sample(5000, step=step, return_inferencedata=True)
    trace = pm.sample(5000, step=step, return_inferencedata=False)

In [ ]:
# Summarize the results
az.summary(trace_for_summary)
az.plot_posterior(trace_for_summary)

In [ ]:
delta_rho_sample = trace['delta_rho']
l_sample = trace['l']
W_sample = trace['W']
N = delta_rho_sample.shape[0]
l_mean = l_sample.sum()/N
delta_rho_mean = delta_rho_sample.sum()/N
W_mean = W_sample.sum()/N

print("delta_rho_mean: ", delta_rho_mean)
print("l_mean: ", l_mean)
print("W_mean: ", W_mean)

expected_u_T_0, expected_u_0 = Foo0(ts, delta_rho_mean, l_mean, W_mean)
expected_u_T_1, expected_u_1 = Foo1(ts, delta_rho_mean, l_mean, W_mean)

fig = plt.figure(tight_layout=True, figsize=(7, 6))
gs = gridspec.GridSpec(1, 1)
ax1 = fig.add_subplot(gs[0, 0])
# ax1.plot(ts/ma, (results['u_T'] + results['u'])/cm_per_yr, 'b', label="hinge retreat")
ax1.plot(ts_ma, u_cm_yr_0, 'r.', label="sliding velocity (observed)")
ax1.plot(ts_ma, expected_u_0 / cm_per_yr, 'b.', label="sliding velocity (predicted))")
ax1.plot(ts_ma, u_T_cm_yr_0, 'ro', label="net trench velocity (observed)")
ax1.plot(ts_ma, expected_u_T_0/cm_per_yr, 'bo', label="net trench velocity (predicted)")
ax1.plot(ts_ma, u_T_cm_yr_1, 'r*', label="net trench velocity (observed)")
ax1.plot(ts_ma, expected_u_T_1/cm_per_yr, 'b*', label="net trench velocity (predicted)")